In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('teknium/Replit-v1-CodeInstruct-3B', trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained('teknium/Replit-v1-CodeInstruct-3B', trust_remote_code=True)

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [24]:
prefix = """"
### Instruction:
I want to refactor error types for a function from error to cerr.Cerr. Refactor
all GRPC handlers to return cerr.Cerr instead of error. Do not return any text
other than the refactored text.

Perform this refactor by changing type signatures as follows:

Before:
```
func (s *Server) X(ctx context.Context, req *Y) (*Z, error)
```
After:
```
func (s *Server) X(ctx context.Context, req *Y) (*Z, cerr.Cerr)
```

Also modify errors so they're returning using cerr wrappers.
You have access to the following constructors:

package cerr
// New returns an error using the given explanation. It should only be used
// if you do not have a source error to wrap.
New(ctx context.Context, explanation string) Cerr
// Wrap wraps an error returned by another function using the given explanation.
// It should be used if you have an error returned by another function to wrap.
Wrap(ctx context.Context, explanation string, err error) Cerr

Examples:

Validation errors:

Before:
```
if len(users) == 0 {
    return nil, errors.New("cannot have no users")
}
```
After:
```
if len(users) == 0 {
    return nil, cerr.New(ctx, "cannot have no users")
}
```

Errors returned by other library:

Before:
```
_, err := someOp()
if err != nil {
    return nil, err
}
```
After:
```
_, err := someOp()
if err != nil {
    return nil, cerr.Wrap(ctx, "op failed", err)
}
```

### Input:
"""

func_test = """
func (s *Server) AddMethods(ctx context.Context, req *AddRequest) (*AddResponse, error) {
  res, err := add(req.X, req.Y)
  if err != nil {
    return nil, err
  }
  return &AddResponse{res}, nil
}

func add (x, y int) (int, error) {
  return x + y, nil
}
""".strip()

imports_test = """
import github.com/test/hi
import github.com/cockroachdb/errors
""".strip()

ret_test = """
  res, err := validateAndCheckLogin(req.X, req.Y)
  if err != nil {
    return nil, err
  }
""".strip()

prompt = prefix + ret_test + "\n\n###Response\n" 


x = tokenizer.encode(prompt, return_tensors='pt')
y = model.generate(
    x,
    max_new_tokens=128,
    do_sample=True,
    use_cache=True,
    temperature=0.2,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id,
)

# decoding, clean_up_tokenization_spaces=False to ensure syntactical correctness
generated_code = tokenizer.decode(y[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(generated_code)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


"
### Instruction:
I want to refactor error types for a function from error to cerr.Cerr. Refactor
all GRPC handlers to return cerr.Cerr instead of error. Do not return any text
other than the refactored text.

Perform this refactor by changing type signatures as follows:

Before:
```
func (s *Server) X(ctx context.Context, req *Y) (*Z, error)
```
After:
```
func (s *Server) X(ctx context.Context, req *Y) (*Z, cerr.Cerr)
```

Also modify errors so they're returning using cerr wrappers.
You have access to the following constructors:

package cerr
// New returns an error using the given explanation. It should only be used
// if you do not have a source error to wrap.
New(ctx context.Context, explanation string) Cerr
// Wrap wraps an error returned by another function using the given explanation.
// It should be used if you have an error returned by another function to wrap.
Wrap(ctx context.Context, explanation string, err error) Cerr

Examples:

Validation errors:

Before:
```
if len(us